In [ ]:
import cv2
import numpy as np
import imutils
from datetime import datetime
import os

def draw_contours_and_boxes(frame, contours, min_area=500, draw_contour=True, draw_box=True):
    boxes = []
    for c in contours:
        if cv2.contourArea(c) < min_area:
            continue
        x, y, w, h = cv2.boundingRect(c)
        boxes.append((x, y, w, h))
        if draw_box:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        if draw_contour:
            cv2.drawContours(frame, [c], -1, (0, 0, 255), 1)
    return frame, boxes

def ensure_dir(path):
    d = os.path.dirname(path)
    if d and not os.path.exists(d):
        os.makedirs(d, exist_ok=True)

def process_video(source=0,
                  output_path=None,
                  show=True,
                  min_area=800,
                  resize_width=600,
                  history=500,
                  varThreshold=16,
                  detectShadows=True):
    """Process a webcam or video file and draw contours around moving objects.

    Args:
        source: 0 (default) for webcam, or path to video file.
        output_path: if given, saves processed video to this file.
        show: whether to display frames in a window.
        min_area: minimum contour area to consider (in pixels).
        resize_width: width to resize frames for consistent processing speed.
        history, varThreshold, detectShadows: parameters for MOG2 background subtractor.
    """
    # Create background subtractor
    backSub = cv2.createBackgroundSubtractorMOG2(history=history, varThreshold=varThreshold, detectShadows=detectShadows)

    cap = cv2.VideoCapture(source)
    if not cap.isOpened():
        raise RuntimeError(f"Unable to open video source: {source}")

    writer = None
    if output_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        # We'll initialize writer after reading first frame to get frame size

    try:
        while True:
            grabbed, frame = cap.read()
            if not grabbed:
                break

            # optional resize for speed
            frame = imutils.resize(frame, width=resize_width)

            # Preprocess frame: convert to gray, blur
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (5, 5), 0)

            # Apply background subtractor to get foreground mask
            fg_mask = backSub.apply(gray)

            # Morphological operations to reduce noise
            fg_mask = cv2.erode(fg_mask, None, iterations=1)
            fg_mask = cv2.dilate(fg_mask, None, iterations=2)

            # Threshold to binarize (remove shadows if detectShadows=True -> shadows are 127)
            _, thresh = cv2.threshold(fg_mask, 200, 255, cv2.THRESH_BINARY)

            # Find contours
            contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Draw contours and bounding boxes
            output_frame = frame.copy()
            output_frame, boxes = draw_contours_and_boxes(output_frame, contours, min_area=min_area)

            # Overlay info
            cv2.putText(output_frame, f"Detections: {len(boxes)}", (10, 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv2.putText(output_frame, datetime.now().strftime('%Y-%m-%d %H:%M:%S'), (10, output_frame.shape[0]-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

            # Initialize writer if needed
            if output_path and writer is None:
                ensure_dir(output_path)
                h, w = output_frame.shape[:2]
                writer = cv2.VideoWriter(output_path, fourcc, 20, (w, h))

            if writer is not None:
                writer.write(output_frame)

            if show:
                cv2.imshow('Motion Contours', output_frame)
                cv2.imshow('Foreground Mask', thresh)

                # Press 'q' to quit
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
    finally:
        cap.release()
        if writer is not None:
            writer.release()
        cv2.destroyAllWindows()

    print('Processing finished.')
process_video('./1625972-hd_1920_1080_25fps.mp4')



